In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import gc
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import fbeta_score

In [3]:
train_classes = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/train_classes.csv')
train_classes.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [4]:
#splitting the train labels
labels = train_classes['tags'].apply(lambda x: x.split(' '))

In [5]:
#getting a dictionary of unique labels
from collections import Counter
labels_count = Counter(label for lbs in labels for label in lbs)

In [6]:
tag_list = list(labels_count.keys())
tag_values = list(labels_count.values())

In [7]:
len(tag_list)

17

In [8]:
test_classes = pd.read_csv('/kaggle/input/planets-dataset/planet/planet/sample_submission.csv')
test_classes.head()

,image_name,tags
0,test_0,primary clear agriculture road water
1,test_1,primary clear agriculture road water
2,test_2,primary clear agriculture road water
3,test_3,primary clear agriculture road water
4,test_4,primary clear agriculture road water


In [10]:
#splitting the test labels
labels = train_classes['tags'].apply(lambda x: x.split(' '))

In [11]:
#getting a dictionary for the test labels
labels_count = Counter(label for lbs in labels for label in lbs)

In [12]:
test_tag_list = list(labels_count.keys())
test_tag_values = list(labels_count.values())

In [13]:
len(test_tag_list)

17

In [14]:
label_map = {i:j for j, i in enumerate(tag_list)}
label_map

{'haze': 0,
 'primary': 1,
 'agriculture': 2,
 'clear': 3,
 'water': 4,
 'habitation': 5,
 'road': 6,
 'cultivation': 7,
 'slash_burn': 8,
 'cloudy': 9,
 'partly_cloudy': 10,
 'conventional_mine': 11,
 'bare_ground': 12,
 'artisinal_mine': 13,
 'blooming': 14,
 'selective_logging': 15,
 'blow_down': 16}

In [15]:
X_train, Y_train = [], []
for img, label in tqdm(train_classes.values, miniters = 1000):
    target = np.zeros(17)
    for tag in label.split(' '):
        target[label_map[tag]]=1
    Y_train.append(target)

100%|██████████| 40479/40479 [00:00<00:00, 181863.71it/s]


In [16]:
#gets all the jpg paths for the train-jpg file
import glob
train_images_path = glob.glob('/kaggle/input/planets-dataset/planet/planet/train-jpg/*.jpg')

In [17]:
import cv2
for file in train_images_path:
     X_train.append(cv2.resize(cv2.imread(file), (64,64)))

In [18]:
import sys
sys.getsizeof(X_train)

361296

In [20]:
#changing the data type to an array and also scaling
x_train = np.array(X_train, np.float16)/255
y_train = np.array(Y_train, np.uint8)

In [21]:
train_X, x_val, train_Y, y_val = train_test_split(x_train, y_train, test_size = 0.2, shuffle = True, random_state = 1)

print(train_X.shape, train_Y.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [22]:
del(X_train, Y_train)

In [23]:
from keras import backend as K
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [23]:
#first model using CNN
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),padding='same', activation='relu', input_shape=(64, 64,3)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
        
model.add(Conv2D(128, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
        
model.add(Conv2D(256, kernel_size=(3, 3),padding='same', activation='relu'))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
        
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(17, activation='sigmoid'))

In [29]:
epochs = 20
learn_rate = 0.0001
opt  = optimizers.Adam(lr=learn_rate)
model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])
callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]
#    ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0)] #save the weights of the best performing model

model.fit(x = train_X, y= train_Y, validation_data=(x_val, y_val),batch_size=128,verbose=2, epochs=epochs)
p_val = model.predict(x_val, batch_size = 32, verbose=2)
print(fbeta_score(y_val, np.array(p_val) > 0.2, beta=2, average='samples')) #Check the model performance on the validation set

Epoch 1/20
253/253 - 8s - loss: 0.2578 - accuracy: 0.8590 - val_loss: 0.2534 - val_accuracy: 0.8678
Epoch 2/20
253/253 - 7s - loss: 0.2578 - accuracy: 0.8590 - val_loss: 0.2534 - val_accuracy: 0.8678
Epoch 3/20
253/253 - 7s - loss: 0.2578 - accuracy: 0.8590 - val_loss: 0.2534 - val_accuracy: 0.8678
Epoch 4/20
253/253 - 7s - loss: 0.2578 - accuracy: 0.8590 - val_loss: 0.2533 - val_accuracy: 0.8678
Epoch 5/20
253/253 - 7s - loss: 0.2575 - accuracy: 0.8590 - val_loss: 0.2534 - val_accuracy: 0.8678
Epoch 6/20
253/253 - 7s - loss: 0.2576 - accuracy: 0.8590 - val_loss: 0.2534 - val_accuracy: 0.8678
Epoch 7/20
253/253 - 7s - loss: 0.2575 - accuracy: 0.8590 - val_loss: 0.2533 - val_accuracy: 0.8678
Epoch 8/20
253/253 - 7s - loss: 0.2577 - accuracy: 0.8590 - val_loss: 0.2534 - val_accuracy: 0.8678
Epoch 9/20
253/253 - 7s - loss: 0.2574 - accuracy: 0.8590 - val_loss: 0.2534 - val_accuracy: 0.8678
Epoch 10/20
253/253 - 7s - loss: 0.2575 - accuracy: 0.8590 - val_loss: 0.2534 - val_accuracy: 0.8678

In [24]:
gc.collect()

119

In [26]:
#second model using transfer learning 
from keras.applications.resnet import ResNet152
from keras.layers import BatchNormalization
base_model = ResNet152(include_top=False, weights='imagenet', input_shape=(64, 64, 3), pooling='avg')
#base_model.trainable = False # Freeze the base_model

model_2 = keras.models.Sequential()
model_2.add(BatchNormalization(input_shape=(64, 64, 3)))
model_2.add(base_model)
model_2.add(Flatten())
model_2.add(Dense(512, activation = "relu"))
model_2.add(Dropout(0.5))
model_2.add(Dense(17, activation = "sigmoid"))
model_2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 64, 64, 3)         12        
_________________________________________________________________
resnet152 (Functional)       (None, 2048)              58370944  
_________________________________________________________________
flatten (Flatten)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 17)                8721      
Total params: 59,428,765
Trainable params: 59,277,335
Non-trainable params: 151,430
____________________________________

In [28]:
from keras.optimizers import Adam
learning_rate = 0.0001
lr_decay = 1e-4
batch_size=64
epochs=20
opt = Adam(lr=learning_rate, decay=lr_decay)

In [31]:
model_2.compile(loss='binary_crossentropy',optimizer=opt,metrics=[fbeta])
callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]
#    ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0)] #save the weights of the best performing model

model_2.fit(x = train_X, y= train_Y, validation_data=(x_val, y_val),batch_size=128,verbose=2, epochs=epochs,callbacks=callbacks,shuffle=True)
        
p_val = model_2.predict(x_val, batch_size = 32, verbose=2)
print(fbeta_score(y_val, np.array(p_val) > 0.2, beta=2, average='samples')) #Check the model performance on the validation set

Epoch 1/20
253/253 - 58s - loss: 0.3161 - fbeta: 0.5747 - val_loss: 0.2697 - val_fbeta: 0.6110
Epoch 2/20
253/253 - 54s - loss: 0.2693 - fbeta: 0.5938 - val_loss: 0.2585 - val_fbeta: 0.6110
Epoch 3/20
253/253 - 54s - loss: 0.2591 - fbeta: 0.5994 - val_loss: 0.2632 - val_fbeta: 0.6109
Epoch 4/20
253/253 - 54s - loss: 0.2436 - fbeta: 0.6084 - val_loss: 0.2967 - val_fbeta: 0.6061
253/253 - 8s
0.6777699392847032


In [38]:
gc.collect()

40

In [34]:
test_images_1 = glob.glob('/kaggle/input/planets-dataset/planet/planet/test-jpg/*.jpg')
test_images_2 = glob.glob('/kaggle/input/planets-dataset/test-jpg-additional/test-jpg-additional/*.jpg')

In [35]:
X_test=[]
for file in test_images_1:
     X_test.append(cv2.resize(cv2.imread(file), (64,64)))
for file in test_images_2:
     X_test.append(cv2.resize(cv2.imread(file), (64,64)))

In [37]:
x_test = np.array(X_test, np.float16)/255

In [40]:
gc.collect()

20

In [ ]:
prediction = model.predict(x_test, batch_size = 128)

In [42]:
prediction_2 = model_2.predict(x_test, batch_size = 128)

In [43]:
pred_tags1 = pd.DataFrame(prediction_2)
pred_tags1 = pred_tags1.apply(lambda x: ' '.join(np.array(tag_list)[x > 0.5]), axis=1)

# converting the predictions of the first 40669 to a dataframe
result = pd.DataFrame({'image_name': test_classes['image_name'], 'tags': pred_tags1})

In [45]:
result.to_csv('final_submission', index = False)

In [46]:
pd.read_csv('final_submission')

,Unnamed: 0,image_name,tags
0,0,test_0,primary clear
1,1,test_1,primary clear
2,2,test_2,primary clear
3,3,test_3,primary agriculture clear
4,4,test_4,primary
...,...,...,...
61186,61186,file_9995,primary clear
61187,61187,file_9996,primary clear
61188,61188,file_9997,primary clear
61189,61189,file_9998,primary clear
